<div style='font-size:2.3em;font-weight:bold'><center>Memory Module: Results and Methods</center></div><br>

<a target="_blank" href='http://i.dailymail.co.uk/i/pix/2014/10/07/1412693459422_wps_70_23_Oct_2012_Hippocampus_C.jpg'><img height= "78" width="102" align="left" src='http://i.dailymail.co.uk/i/pix/2014/10/07/1412693459422_wps_70_23_Oct_2012_Hippocampus_C.jpg'></a>
<br><br><br><br>(1)
#Aims#
First Stage Goal: Simulate to a partial extent neural activity involved in memory processes.  Specifically, create a neural network that recreates the spiking activity recorded from in vivo hippocampus regions of rats.  Later this network can be used to recreate activity changes that occur in memory and perhaps other recorded tasks.

Future goals are <a target="_blank" href='/github/nmsutton/MemoryModule/blob/master/memory_module_outline.ipynb'>included here</a>.

##<a target="_blank" href='http://www.github.com/nmsutton/MemoryModule'>Code for the project is in on Github here.</a>##
<br>
<font color="darkblue" size=+1><b>**UPDATE**</b>: The project has successfully been converted into using GPU computing with <a href="http://www.socsci.uci.edu/~jkrichma/CARLsim/">CARLSim</a> in C++.  The python original version using <a href="http://www.nest-simulator.org/">NEST</a> is included in the python_version folder.</font>

#Results#

EC3->EC5 feedforward layers spiking
<br>32803/31967.975 = <b>97.45% accuracy</b>

EC5->CA1 feedforward layers spiking
<br>155615/149667.83 = <b>96.18% accuracy</b>

Overall accuracy
<br>97.45% \* 96.18% = <b>93.73%</b>

See <a target="_blank" href='/github/nmsutton/MemoryModule/blob/master/analyses_descriptions/results_calculations.ipynb'>results_calculations</a> for more specific details.

#Collecting recordings#
<a target="_blank" href='http://www.frontiersin.org/files/Articles/99155/fnana-08-00091-HTML/image_m/fnana-08-00091-g010.jpg'><img height= "78" width="102" align="left" src='http://www.frontiersin.org/files/Articles/99155/fnana-08-00091-HTML/image_m/fnana-08-00091-g010.jpg'></a>
<br><br><br><br><br>(2)<br>
Hippocampus rat recordings were extracted from open access data <a target="_blank" href='http://crcns.org/data-sets/hc/hc-3'>here</a>.  Existing code for importing the data was converted to octave and the octave commands used were as follows:
<br>[T,G,Map,Par]=LoadCluRes({File},{Electrode});
<br>Where {File} = base data file, {Electrode} = recording probe number, T = spike times, G = cluster representing putitive neuron
<br>Examples of use are in code below.

1.) Numbers of putitive neurons were found with <a target="_blank" href='/github/nmsutton/MemoryModule/blob/master/analyses_descriptions/neuron_counts.m'>neuron_counts.m</a>.  Example output is in <a target="_blank" href='/github/nmsutton/MemoryModule/blob/master/analyses_descriptions/example_analyses.ipynb'>example_analyses</a>.

2.) Amounts of spikes for given time ranges were found using <a target="_blank" href='/github/nmsutton/MemoryModule/blob/master/analyses_descriptions/count_spikes.m'>count_spikes.m</a>, spiking from all electrodes in a dataset section were found with <a target="_blank" href='/github/nmsutton/MemoryModule/blob/master/analyses_descriptions/elect_spikes.m'>elect_spikes.m</a>.  The time range chosen for analysis was the first 2000000 ms.

3.) Which brain region the parts of the data were derived from were found using the <a target="_blank" href='http://crcns.org/data-sets/hc/hc-3'>/hc-3/hc3-metadata-tables/hc3-tables.xlsx</a> document.
<br>For example (e1 = electrode 1):
<br>topdir     |  animal  |  e1 |  e2  |  e3  |  e4
<br>ec012ec.11 |  ec012   |  EC3  |  EC3  |  EC5  |  EC5

From the 3 steps above spike rates of neurons and info. about brain region included for the neurons was gathered.

#Neural Network Modeling#
###This <a href="http://www.socsci.uci.edu/~jkrichma/CARLsim/">CARLSim</a> simulation <a target="_blank" href='https://github.com/nmsutton/MemoryModule/blob/master/main_mem_mod_gpu.cpp'>source code is here</a>.###
A spiking neural network was created in <a href="http://www.socsci.uci.edu/~jkrichma/CARLsim/">CARLSim</a> to simulate the activity of the hippocampus neurons.  

<a target="_blank" href='http://www.ebi.ac.uk/biomodels/ModelMonth/2010-05/figure1.png'><img height= "150" width="200" align="left" src='http://www.ebi.ac.uk/biomodels/ModelMonth/2010-05/figure1.png'></a>
<br><br><br><br><br><br><br>(3)<br>
Izhikevich neurons were used along with example values reported for pyramidal neurons.  That neuron type comprised the main portion of the data used here.

The neuron spiking rates were normalized to create each representitive group containing the same estimated 
number of neurons.  For example the most neurons per a brain regoin recorded were 30 and due to that brain
regions with less than 30 had their spike rates multiplied to increase the representitve rate to 30 neurons.
This allowed for a more straitforward mathmatical way to connect neural net layers representing regions.

##<ul><li>Creating Spiking in the Network</li></ul>##
Feed forward synapses were created that represent spiking in layers leading to spiking in later ones.  Ratios of spiking between layers were found by dividing each layer's spiking rate by it's prior one.  This created the goal effects to simulate with synapses between layers.

Greater detail was achived by separating firing rates into individual subsection groups based on experimental results.  For example a group of 7 neurons were recorded in the EC5 layer as firing an estimated 423.9286 spikes and 7 EC3 neurons fired 284.1742 .  Therefore the spike ratio between layers was 423.9286 / 284.17424 = 1.4918 .

##<ul><li>Optimizing Synapse Weights</li></ul>##
<a target="_blank" href='https://github.com/nmsutton/MemoryModule/blob/master/main_mem_mod_gpu.cpp'>create_syn_weights()</a> is used for computing optimized synapse weights, it uses parameter values based on multiple polynomial regression fitting.

A multiple polynomial regression tool was used to automatically determine what synapse weights should be based on initial firing rate and ratio between layers wanted.  Testing revealed successful values for synapse weights and a data set was created to automatically predict weights using regression for future analyses.  Data sets are in <a target="_blank" href='/github/nmsutton/MemoryModule/blob/master/analyses_descriptions/synapse_weights_data.ipynb'>synapse_weights_data</a>.


##<ul><li>Creating Synapses</li></ul>##
<a target="_blank" href='https://github.com/nmsutton/MemoryModule/blob/master/main_mem_mod_gpu.cpp'>create_syn()</a> generates synapse weights based on multiple parameters such as fire rate ratio between layers and computed synapse weights.

A strategy was chosen due to computational practicality to create synapse weights leading to one layer causing firing in the next by creating a number of synapse connections that corresponds to the ratio targeted.  For example for a 0.5 ratio the initial layer caused spiking in 50% of the second layer's synapses.  It was designated that mainly the only spiking in layers was triggered from the synapses connected to them, layers had very little spiking due to an external current except for the first one.

For each ratio interval over 1 (e.g. 2.0, 3.0, 4.0) synapses with a preset delay were added to create a capacity to increase spiking.  Such a system caused a staggered time sequence of spikes firing.  After testing the delay selected was 5ms due to achiving good results.  For example a ratio of 2.3 causes synapses to fire in the later layer 100% for a 0ms delay, 100% for a 5ms delay, and 30% for a 10ms delay.

#Capturing Results#
Firing rate results are measured and reported from spike_monitors objects.

Future work includes plotting of spike raser plots and electrical signal graphs.

Media references:
    1. http://i.dailymail.co.uk/i/pix/2014/10/07/1412693459422_wps_70_23_Oct_2012_Hippocampus_C.jpg
    2. http://www.frontiersin.org/files/Articles/99155/fnana-08-00091-HTML/image_m/fnana-08-00091-g010.jpg
    3. http://www.ebi.ac.uk/biomodels/ModelMonth/2010-05/figure1.png